Grab relevant information for bus stops for a specific route, join the data together.

In [ ]:
import requests
import pandas as pd
import glob
import json
import time
import os
import platform
from analysis_functions import *

with open('config.json', 'r') as f:
    config = json.load(f)

SWIFTLY_API_KEY = config['DEFAULT']['SWIFTLY_API_KEY']
MSSQL_USERNAME = config['DEFAULT']['MSSQL_USERNAME']
MSSQL_PASSWORD = config['DEFAULT']['MSSQL_PASSWORD']

if platform.system() == 'Darwin':
    import pymssql
    connection = pymssql.connect(server='ELTDBPRD\ELTDBPRD', user=MSSQL_USERNAME, password=MSSQL_PASSWORD, database='ACS_13')
elif platform.system() == 'Windows':
    import pyodbc
    connection_string = 'DRIVER={SQL Server};SERVER=ELTDBPRD\ELTDBPRD;DATABASE=ACS_13;UID=%s;PWD=%s' % (MSSQL_USERNAME, MSSQL_PASSWORD)
    connection = pyodbc.connect(connection_string)


# DEBUG = True

days_to_consider = [2,3,4,5,9,10,11,12,16,17,18,19,23,24,25,26,30]

In [ ]:
line_numbers = [ 22, 181,  68,  23,  66,  25,  10, 200, 55, 304,  19,
        64,  72,  70, 180,  62,  71,  57, 121, 168,  26,  60, 102,  27,
        35,  48,  65,  32, 328, 103, 104, 522,  61, 122,  77,  73,  46,
        95, 101,  54,  88,  89,  40,  81,  52,
        53,  58,  31,  63, 323,  82,  47, 120, 182, 201, 330, 140,  49,
        37,  16,  13,  42,  18,  14,  17,  45,  39, 321, 185,  34, 824,
       831, 827, 828, 826, 825, 823,  12, 231, 251]
# NO LR

In [ ]:
for line_number in line_numbers:

    rid_by_stop_df = pull_ridership_by_stop(line_number)
    rid_by_stop_df.head()

    df_dwell_runtime, df_stop_path_length, df_min_travel_time = dwell_runtime(line_number, days_to_consider)

    rid_dwell = pd.merge(pd.merge(pd.merge(rid_by_stop_df,df_dwell_runtime,how='outer'),df_stop_path_length, how='outer'),df_min_travel_time, how='outer')

    stops_visited_counts, trips_sampled_count = stop_frequency_percent(connection, line_number, days_to_consider, date_range= "'2017-10-01' and '2017-11-1'")
    del stops_visited_counts['current_route_id']
    del trips_sampled_count['current_route_id']

    bus_df_frequency = pd.merge(pd.merge(rid_dwell, stops_visited_counts, how="outer"),trips_sampled_count, how="outer")
    # stop_frequency['percent_stopped'] = (stop_frequency['number_of_times_stopped']/stop_frequency['total_trips_sampled']).round(2)
    # bus_df_frequency['percent_stopped'] = (bus_df_frequency['number_of_times_stopped'].dividebus_df_frequency['total_trips_sampled']).round(2)
    bus_df_frequency['percent_stopped'] = (bus_df_frequency['number_of_times_stopped'].divide(bus_df_frequency['total_trips_sampled'],fill_value=0)).round(2)

    bus_df_frequency['travel_speed_meters_second'] = (bus_df_frequency['stop_path_length_meters']/bus_df_frequency['travel_time_secs_mean']).round(2)
    bus_df_frequency['travel_speed_miles_per_hour'] = ((bus_df_frequency['stop_path_length_meters']/bus_df_frequency['travel_time_secs_mean'])*2.23694).round(1)

    bus_df_frequency.to_csv("results/bus_stop_data_analysis_dwell_" + str(line_number) + ".csv",index=False)